In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import itertools
import re
from itertools import product
import operator
from functools import reduce
#from pandas import option_context
#%pip install openpyxl
%config InlineBackend.figure_format = 'retina'
plt.style.use('seaborn-pastel')
plt.style.use('seaborn-dark')

/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_97674/1792471945.py:14: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-pastel')
/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_97674/1792471945.py:15: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-dark')


In [1]:
import matplotlib
from matplotlib.colors import ListedColormap
from matplotlib.colors import LinearSegmentedColormap
matplotlib.style.use('dark_background')
color_list = ['white','skyblue','pink', 'tomato', 'slateblue', 'wheat', 'lightgrey', 'limegreen', 'royalblue', 'yellowgreen', 'lightseagreen', 'orchid', 'orange']
cmap = LinearSegmentedColormap.from_list("",color_list)

In [2]:
# Import all the google connexion packages
import os
import google.auth
from google.cloud import bigquery
from google.auth import impersonated_credentials

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/a20015173/.config/gcloud/legacy_credentials/ekaterina.rezanovich@adeo.com/adc.json"
client_1 = bigquery.Client(project = 'ccdp-transverse-data-dev')

In [4]:
# questions data
filepath_1 = "./datasets/questions.parquet"
if not os.path.isfile(filepath_1):

    query = """
    select t1.survey_id, t1.name , t2.question_id, t2.name, t2.content
    from `dfdp-customer-exp-mt-prod.customer_experience_measure_bu001.client_experience_survey` as t1
    left join `dfdp-customer-exp-mt-prod.customer_experience_measure_bu001.client_experience_question` as t2 on t2.survey_id = t1.survey_id
    where t2.question_id in ('SV_81jKyfOtepO6Weq__QID1_NPS_GROUP', 'SV_4OUB8z9FwEgBiU6__QID1_NPS_GROUP', 'SV_5nFeSHRTsx1yxWC__QID1_NPS_GROUP', 'SV_77FLHcnyIyMM07Q__QID1_NPS_GROUP', 'SV_bj9J6UgbM2yIATQ__QID13_NPS_GROUP',
    'SV_2fA21joHLvPEZoi__QID1_NPS_GROUP', 'SV_5nVqXD824PZXj14__QID1_NPS_GROUP')"""

    dl_data = pd.read_gbq(query=query, project_id="ccdp-transverse-data-dev")
    dl_data.to_parquet(filepath_1)

df_q = pd.read_parquet(filepath_1)

In [5]:
df_q.head(2)

,survey_id,name,question_id,name_1,content
0,SV_77FLHcnyIyMM07Q,LMFR - CLICK & COLLECT - PROD,SV_77FLHcnyIyMM07Q__QID1_NPS_GROUP,QID1_NPS_GROUP,"Suite à votre retrait marchandise, recommander..."
1,SV_4OUB8z9FwEgBiU6,LMFR - INSTALLATION - PROD,SV_4OUB8z9FwEgBiU6__QID1_NPS_GROUP,QID1_NPS_GROUP,Vous avez récemment fait appel aux services de...


In [6]:
# answers data
filepath_2 = "./datasets/answers.parquet"
if not os.path.isfile(filepath_2):

    query = """
select t3.answer_id, t3.feedback_id, t3.question_id, t3.score, t3.score_category, t3.comment,
t4.touchpoint_id, t4.store_code, t4.user_id, t4.user_id_type, cast(t4.date as date) as answer_date
from `dfdp-customer-exp-mt-prod.customer_experience_measure_bu001.client_experience_answer` as t3
left join `dfdp-customer-exp-mt-prod.customer_experience_measure_bu001.client_experience_feedback` as t4 on t4.feedback_id = t3.feedback_id
where t3.question_id in ('SV_81jKyfOtepO6Weq__QID1_NPS_GROUP', 'SV_4OUB8z9FwEgBiU6__QID1_NPS_GROUP', 'SV_5nFeSHRTsx1yxWC__QID1_NPS_GROUP', 'SV_77FLHcnyIyMM07Q__QID1_NPS_GROUP', 'SV_bj9J6UgbM2yIATQ__QID13_NPS_GROUP',
'SV_2fA21joHLvPEZoi__QID1_NPS_GROUP', 'SV_5nVqXD824PZXj14__QID1_NPS_GROUP')
    """

    dl_data = pd.read_gbq(query=query, project_id="ccdp-transverse-data-dev")
    dl_data.to_parquet(filepath_2)

df_a = pd.read_parquet(filepath_2)

In [12]:
df_nps_all = df_a.merge(df_q, left_on = 'question_id', right_on = 'question_id', how='left')

In [13]:
del df_nps_all['name_1'], df_nps_all['user_id_type'],  df_nps_all['content']

In [14]:
df_nps_all.head(2)

,answer_id,feedback_id,question_id,score,score_category,comment,touchpoint_id,store_code,user_id,answer_date,survey_id,name
0,R_111w3BVzUYwknmY__QID1_NPS_GROUP,R_111w3BVzUYwknmY,SV_81jKyfOtepO6Weq__QID1_NPS_GROUP,8,Passif,"Bonjour, \nJe suis entièrement satisfait de ...",SV_81jKyfOtepO6Weq,<NA>,10217876,2023-08-20,SV_81jKyfOtepO6Weq,LMFR - Client SAV - PROD
1,R_3htf0IgZ0ecDb8u__QID1_NPS_GROUP,R_3htf0IgZ0ecDb8u,SV_81jKyfOtepO6Weq__QID1_NPS_GROUP,10,Promoteur,None,SV_81jKyfOtepO6Weq,<NA>,54974624,2023-08-16,SV_81jKyfOtepO6Weq,LMFR - Client SAV - PROD


In [15]:
# filter only nps with user_id
df_nps = df_nps_all[df_nps_all['user_id'].notna()]

In [20]:
df_nps_all[df_nps_all['user_id'].isna()]['name'].unique()

array(['LMFR - INSTALLATION - PROD', 'LMFR - WEBSITE VISIT - PROD',
       'LMFR - WEBSITE PURCHASE - PROD', 'LMFR - STORE PURCHASE - PROD',
       'LMFR - CLICK & COLLECT - PROD', 'LMFR - CUSTOMER CARE - PROD'],
      dtype=object)

In [16]:
df_nps['name'].unique()

array(['LMFR - Client SAV - PROD', 'LMFR - INSTALLATION - PROD',
       'LMFR - WEBSITE PURCHASE - PROD', 'LMFR - STORE PURCHASE - PROD',
       'LMFR - CLICK & COLLECT - PROD', 'LMFR - CUSTOMER CARE - PROD'],
      dtype=object)

In [21]:
# first and last TP of the customer
df_nps_dates = df_nps.groupby('user_id')['answer_date'].agg(['min', 'max']).reset_index()
df_nps_dates.columns = ['uid', 'first_nps_date', 'last_nps_date']
df_nps_dates.head()

,user_id,min,max
0,1000000,2023-03-15,2023-06-18
1,10000024,2022-04-09,2022-04-09
2,1000004,2022-11-13,2022-11-13
3,1000011,2022-07-11,2023-03-10
4,10000188,2023-05-30,2023-05-30


In [24]:
df_nps_1 = df_nps.merge(df_nps_dates, left_on='user_id', right_on='uid', how='left')
del df_nps_1['uid']
df_nps_1.head(2)

,answer_id,feedback_id,question_id,score,score_category,comment,touchpoint_id,store_code,user_id,answer_date,survey_id,name,uid,first_nps_date,last_nps_date
0,R_111w3BVzUYwknmY__QID1_NPS_GROUP,R_111w3BVzUYwknmY,SV_81jKyfOtepO6Weq__QID1_NPS_GROUP,8,Passif,"Bonjour, \nJe suis entièrement satisfait de ...",SV_81jKyfOtepO6Weq,<NA>,10217876,2023-08-20,SV_81jKyfOtepO6Weq,LMFR - Client SAV - PROD,10217876,2022-04-09,2023-08-20
1,R_3htf0IgZ0ecDb8u__QID1_NPS_GROUP,R_3htf0IgZ0ecDb8u,SV_81jKyfOtepO6Weq__QID1_NPS_GROUP,10,Promoteur,None,SV_81jKyfOtepO6Weq,<NA>,54974624,2023-08-16,SV_81jKyfOtepO6Weq,LMFR - Client SAV - PROD,54974624,2023-08-16,2023-08-16


In [26]:
# nb FB of the customer
df_nps_nb_fb = df_nps.groupby('user_id')['feedback_id'].nunique().reset_index()
df_nps_nb_fb.columns = ['uid', 'nb_fb']
df_nps_nb_fb.head()

,uid,nb_fb
0,1000000,2
1,10000024,1
2,1000004,1
3,1000011,2
4,10000188,1


In [28]:
df_nps_2 = df_nps_1.merge(df_nps_nb_fb, left_on='user_id', right_on='uid', how='left')
del df_nps_2['uid']
df_nps_2.head(2)

,answer_id,feedback_id,question_id,score,score_category,comment,touchpoint_id,store_code,user_id,answer_date,survey_id,name,first_nps_date,last_nps_date,nb_fb
0,R_111w3BVzUYwknmY__QID1_NPS_GROUP,R_111w3BVzUYwknmY,SV_81jKyfOtepO6Weq__QID1_NPS_GROUP,8,Passif,"Bonjour, \nJe suis entièrement satisfait de ...",SV_81jKyfOtepO6Weq,<NA>,10217876,2023-08-20,SV_81jKyfOtepO6Weq,LMFR - Client SAV - PROD,2022-04-09,2023-08-20,14
1,R_3htf0IgZ0ecDb8u__QID1_NPS_GROUP,R_3htf0IgZ0ecDb8u,SV_81jKyfOtepO6Weq__QID1_NPS_GROUP,10,Promoteur,None,SV_81jKyfOtepO6Weq,<NA>,54974624,2023-08-16,SV_81jKyfOtepO6Weq,LMFR - Client SAV - PROD,2023-08-16,2023-08-16,1


In [31]:
# 0 for detractor, 1 for passive, 2 for promotor
def fb_type(df):
    if df['score'] <= 6 :
        return 0
    elif df['score'] >= 9:
        return 2
    else:
        return 1

In [33]:
df_nps_2['score'].unique()

<IntegerArray>
[8, 10, 9, 4, 0, 3, 6, 5, 1, 7, 2, <NA>]
Length: 12, dtype: Int64

In [34]:
df_nps_3 = df_nps_2[df_nps_2['score'].notna()]

In [36]:
df_nps_3['fb_type'] = df_nps_3.apply(fb_type, axis=1)
df_nps_3.head(2)

/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_46488/3604727486.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nps_3['fb_type'] = df_nps_3.apply(fb_type, axis=1)


,answer_id,feedback_id,question_id,score,score_category,comment,touchpoint_id,store_code,user_id,answer_date,survey_id,name,first_nps_date,last_nps_date,nb_fb,fb_type
0,R_111w3BVzUYwknmY__QID1_NPS_GROUP,R_111w3BVzUYwknmY,SV_81jKyfOtepO6Weq__QID1_NPS_GROUP,8,Passif,"Bonjour, \nJe suis entièrement satisfait de ...",SV_81jKyfOtepO6Weq,<NA>,10217876,2023-08-20,SV_81jKyfOtepO6Weq,LMFR - Client SAV - PROD,2022-04-09,2023-08-20,14,1
1,R_3htf0IgZ0ecDb8u__QID1_NPS_GROUP,R_3htf0IgZ0ecDb8u,SV_81jKyfOtepO6Weq__QID1_NPS_GROUP,10,Promoteur,None,SV_81jKyfOtepO6Weq,<NA>,54974624,2023-08-16,SV_81jKyfOtepO6Weq,LMFR - Client SAV - PROD,2023-08-16,2023-08-16,1,2


In [41]:
# types of the customer fb
df_nps_fb_typ = df_nps_3.pivot_table(index='user_id', columns='fb_type', values='feedback_id', aggfunc='nunique').reset_index()
df_nps_fb_typ.columns = ['uid', '0', '1', '2']
df_nps_fb_typ = df_nps_fb_typ.fillna(0)
df_nps_fb_typ.head()

In [44]:
def fb_type_res(df):
    if df['0']==0 and df['1']==0:
        return "only positive"
    elif df['0']==0 and df['2']==0:
        return "only passive"
    elif df['1']==0 and df['2']==0:
        return "only negative"
    elif df['0']==0:
        return "passive and positive"
    elif df['1']==0:
        return "negative and positive"  
    elif df['2']==0:
        return "negative and passive"  
    else:
        return "negative, passive, positive"

In [46]:
df_nps_fb_typ['fb_type'] = df_nps_fb_typ.apply(fb_type_res, axis=1)

In [51]:
def fb_type_res_nb(df):
    if (df['0']+df['1']+df['2']) > 1:
        return "mutli"
    else:
        return "mono"


In [52]:
df_nps_fb_typ['fb_nb_type'] = df_nps_fb_typ.apply(fb_type_res_nb, axis=1)

In [56]:
df_nps_fb_typ.columns = ['uid', 'nb_fb_neg', 'nb_fb_pas', 'nb_fb_pos', 'fb_group', 'fb_nb_type']
df_nps_fb_typ.head(2)

,uid,nb_fb_neg,nb_fb_pas,nb_fb_pos,fb_group,fb_nb_type
0,1000000,0.0,0.0,2.0,only positive,mutli
1,10000024,1.0,0.0,0.0,only negative,mono


In [58]:
df_nps_4 = df_nps_3.merge(df_nps_fb_typ, left_on='user_id', right_on='uid', how='left')
del df_nps_4['uid']
df_nps_4.head(2)

,answer_id,feedback_id,question_id,score,score_category,comment,touchpoint_id,store_code,user_id,answer_date,...,name,first_nps_date,last_nps_date,nb_fb,fb_type,nb_fb_neg,nb_fb_pas,nb_fb_pos,fb_group,fb_nb_type
0,R_111w3BVzUYwknmY__QID1_NPS_GROUP,R_111w3BVzUYwknmY,SV_81jKyfOtepO6Weq__QID1_NPS_GROUP,8,Passif,"Bonjour, \nJe suis entièrement satisfait de ...",SV_81jKyfOtepO6Weq,<NA>,10217876,2023-08-20,...,LMFR - Client SAV - PROD,2022-04-09,2023-08-20,14,1,0.0,1.0,13.0,passive and positive,mutli
1,R_3htf0IgZ0ecDb8u__QID1_NPS_GROUP,R_3htf0IgZ0ecDb8u,SV_81jKyfOtepO6Weq__QID1_NPS_GROUP,10,Promoteur,None,SV_81jKyfOtepO6Weq,<NA>,54974624,2023-08-16,...,LMFR - Client SAV - PROD,2023-08-16,2023-08-16,1,2,0.0,0.0,1.0,only positive,mono


In [63]:
df_nps_summary_type = df_nps_4.groupby('fb_nb_type')['user_id'].nunique().reset_index()
df_nps_summary_type['share_%'] = (df_nps_summary_type['user_id'] / df_nps_summary_type['user_id'].sum())*100
df_nps_summary_type

,fb_nb_type,user_id,share_%
0,mono,889117,72.614886
1,mutli,335311,27.385114


In [67]:
df_nps_summary_group = df_nps_4.groupby('fb_group')['user_id'].nunique().reset_index()
df_nps_summary_group['share_%'] = (df_nps_summary_group['user_id'] / df_nps_summary_group['user_id'].sum())*100
df_nps_summary_group = df_nps_summary_group.sort_values(by='share_%', ascending=False)
df_nps_summary_group

,fb_group,user_id,share_%
5,only positive,680303,55.560882
4,only passive,277548,22.667564
3,only negative,129318,10.561503
6,passive and positive,77828,6.356274
1,negative and positive,26855,2.193269
0,negative and passive,21942,1.792020
2,"negative, passive, positive",10634,0.868487


In [69]:
df_nps_summary = df_nps_4.groupby(['fb_nb_type', 'fb_group'])['user_id'].nunique().reset_index()
df_nps_summary['share_%'] = (df_nps_summary['user_id'] / df_nps_summary['user_id'].sum())*100
df_nps_summary = df_nps_summary.sort_values(by=['fb_nb_type','share_%'], ascending=[True, False])
df_nps_summary

,fb_nb_type,fb_group,user_id,share_%
2,mono,only positive,524428,42.830448
1,mono,only passive,246769,20.153819
0,mono,only negative,117920,9.630619
8,mutli,only positive,155875,12.730434
9,mutli,passive and positive,77828,6.356274
7,mutli,only passive,30779,2.513745
4,mutli,negative and positive,26855,2.193269
3,mutli,negative and passive,21942,1.792020
6,mutli,only negative,11398,0.930884
5,mutli,"negative, passive, positive",10634,0.868487


Now there are different ways to check the customers:
1. check the customers jorney according to every group above  
2. check the baskets  

Sales Data

In [5]:
# sales data 2022 1/4, 1P
query_global_1 = """
SELECT sales.Num_Entity as store_id, sales.Date_Sales as cart_date, sales.Num_Product as item_id, sales.Num_ticket as cart_id, 
sales.Nbr_Checkouts as nb_checkouts, 
sales.Num_SalesChannel as sale_channel_id, sales.Num_DeliveryChannel as delivery_channel_id, 
sales.Amt_GrossMerchandiseVolume as amount_to, sales.Qty_Sales as qty, sales.Top_Loyaltycard as top_l_c, sales.client_Number as user_id,
sales.gmv_type as gmv_type, sales_ch.Desc_SalesChannel as sales_channel, delivery.Desc_DeliveryChannel as delivery_channel

FROM `dfdd-performances-benchmark.BenchmarkData.taGMVProductStoreDay` as sales
left join `dfdd-performances-benchmark.BenchmarkData.tdSalesChannel` as sales_ch on sales_ch.Num_SalesChannel = sales.Num_SalesChannel 
left join `dfdd-performances-benchmark.BenchmarkData.tdDeliveryChannel` as delivery on delivery.Num_DeliveryChannel = sales.Num_DeliveryChannel

WHERE Num_BusinessUnit = 1 and client_Number is not null
and (Date_Sales >='2022-01-01' and Date_Sales <'2022-04-01') and gmv_type = "1P" """

df_sd_22_1_1p_q = client_1.query(query=query_global_1)
df_sd_22_1_1p = df_sd_22_1_1p_q.to_dataframe()

In [6]:
# sales data 2022 2/4, 1P
query_global_2 = """
SELECT sales.Num_Entity as store_id, sales.Date_Sales as cart_date, sales.Num_Product as item_id, sales.Num_ticket as cart_id, 
sales.Nbr_Checkouts as nb_checkouts, 
sales.Num_SalesChannel as sale_channel_id, sales.Num_DeliveryChannel as delivery_channel_id, 
sales.Amt_GrossMerchandiseVolume as amount_to, sales.Qty_Sales as qty, sales.Top_Loyaltycard as top_l_c, sales.client_Number as user_id,
sales.gmv_type as gmv_type, sales_ch.Desc_SalesChannel as sales_channel, delivery.Desc_DeliveryChannel as delivery_channel

FROM `dfdd-performances-benchmark.BenchmarkData.taGMVProductStoreDay` as sales
left join `dfdd-performances-benchmark.BenchmarkData.tdSalesChannel` as sales_ch on sales_ch.Num_SalesChannel = sales.Num_SalesChannel 
left join `dfdd-performances-benchmark.BenchmarkData.tdDeliveryChannel` as delivery on delivery.Num_DeliveryChannel = sales.Num_DeliveryChannel

WHERE Num_BusinessUnit = 1 and client_Number is not null
and (Date_Sales >='2022-04-01' and Date_Sales <'2022-07-01') and gmv_type = "1P"
"""

df_sd_22_2_1p_q = client_1.query(query=query_global_2)
df_sd_22_2_1p = df_sd_22_2_1p_q.to_dataframe()

In [7]:
# sales data 2022 3/4, 1P
query_global_3 = """
SELECT sales.Num_Entity as store_id, sales.Date_Sales as cart_date, sales.Num_Product as item_id, sales.Num_ticket as cart_id, 
sales.Nbr_Checkouts as nb_checkouts, 
sales.Num_SalesChannel as sale_channel_id, sales.Num_DeliveryChannel as delivery_channel_id, 
sales.Amt_GrossMerchandiseVolume as amount_to, sales.Qty_Sales as qty, sales.Top_Loyaltycard as top_l_c, sales.client_Number as user_id,
sales.gmv_type as gmv_type, sales_ch.Desc_SalesChannel as sales_channel, delivery.Desc_DeliveryChannel as delivery_channel

FROM `dfdd-performances-benchmark.BenchmarkData.taGMVProductStoreDay` as sales
left join `dfdd-performances-benchmark.BenchmarkData.tdSalesChannel` as sales_ch on sales_ch.Num_SalesChannel = sales.Num_SalesChannel 
left join `dfdd-performances-benchmark.BenchmarkData.tdDeliveryChannel` as delivery on delivery.Num_DeliveryChannel = sales.Num_DeliveryChannel

WHERE Num_BusinessUnit = 1 and client_Number is not null
and (Date_Sales >='2022-07-01' and Date_Sales <'2022-10-01') and gmv_type = "1P"
"""

df_sd_22_3_1p_q = client_1.query(query=query_global_3)
df_sd_22_3_1p = df_sd_22_3_1p_q.to_dataframe()

In [8]:
# sales data 2022 4/4, 1P
query_global_4 = """
SELECT sales.Num_Entity as store_id, sales.Date_Sales as cart_date, sales.Num_Product as item_id, sales.Num_ticket as cart_id, 
sales.Nbr_Checkouts as nb_checkouts, 
sales.Num_SalesChannel as sale_channel_id, sales.Num_DeliveryChannel as delivery_channel_id, 
sales.Amt_GrossMerchandiseVolume as amount_to, sales.Qty_Sales as qty, sales.Top_Loyaltycard as top_l_c, sales.client_Number as user_id,
sales.gmv_type as gmv_type, sales_ch.Desc_SalesChannel as sales_channel, delivery.Desc_DeliveryChannel as delivery_channel

FROM `dfdd-performances-benchmark.BenchmarkData.taGMVProductStoreDay` as sales
left join `dfdd-performances-benchmark.BenchmarkData.tdSalesChannel` as sales_ch on sales_ch.Num_SalesChannel = sales.Num_SalesChannel 
left join `dfdd-performances-benchmark.BenchmarkData.tdDeliveryChannel` as delivery on delivery.Num_DeliveryChannel = sales.Num_DeliveryChannel

WHERE Num_BusinessUnit = 1 and client_Number is not null
and (Date_Sales >='2022-10-01' and Date_Sales <'2023-01-01') and gmv_type = "1P"
"""

df_sd_22_4_1p_q = client_1.query(query=query_global_4)
df_sd_22_4_1p = df_sd_22_4_1p_q.to_dataframe()

In [9]:
# sales data 2023 1/3, 1P
query_global_5 = """
SELECT sales.Num_Entity as store_id, sales.Date_Sales as cart_date, sales.Num_Product as item_id, sales.Num_ticket as cart_id, 
sales.Nbr_Checkouts as nb_checkouts, 
sales.Num_SalesChannel as sale_channel_id, sales.Num_DeliveryChannel as delivery_channel_id, 
sales.Amt_GrossMerchandiseVolume as amount_to, sales.Qty_Sales as qty, sales.Top_Loyaltycard as top_l_c, sales.client_Number as user_id,
sales.gmv_type as gmv_type, sales_ch.Desc_SalesChannel as sales_channel, delivery.Desc_DeliveryChannel as delivery_channel

FROM `dfdd-performances-benchmark.BenchmarkData.taGMVProductStoreDay` as sales
left join `dfdd-performances-benchmark.BenchmarkData.tdSalesChannel` as sales_ch on sales_ch.Num_SalesChannel = sales.Num_SalesChannel 
left join `dfdd-performances-benchmark.BenchmarkData.tdDeliveryChannel` as delivery on delivery.Num_DeliveryChannel = sales.Num_DeliveryChannel

WHERE Num_BusinessUnit = 1 and client_Number is not null
and (Date_Sales >='2023-01-01' and Date_Sales <'2023-04-01') and gmv_type = "1P"
"""

df_sd_23_1_1p_q = client_1.query(query=query_global_5)
df_sd_23_1_1p = df_sd_23_1_1p_q.to_dataframe()

: 

: 

In [ ]:
# sales data 2023 2/3, 1P
query_global_6 = """
SELECT sales.Num_Entity as store_id, sales.Date_Sales as cart_date, sales.Num_Product as item_id, sales.Num_ticket as cart_id, 
sales.Nbr_Checkouts as nb_checkouts, 
sales.Num_SalesChannel as sale_channel_id, sales.Num_DeliveryChannel as delivery_channel_id, 
sales.Amt_GrossMerchandiseVolume as amount_to, sales.Qty_Sales as qty, sales.Top_Loyaltycard as top_l_c, sales.client_Number as user_id,
sales.gmv_type as gmv_type, sales_ch.Desc_SalesChannel as sales_channel, delivery.Desc_DeliveryChannel as delivery_channel

FROM `dfdd-performances-benchmark.BenchmarkData.taGMVProductStoreDay` as sales
left join `dfdd-performances-benchmark.BenchmarkData.tdSalesChannel` as sales_ch on sales_ch.Num_SalesChannel = sales.Num_SalesChannel 
left join `dfdd-performances-benchmark.BenchmarkData.tdDeliveryChannel` as delivery on delivery.Num_DeliveryChannel = sales.Num_DeliveryChannel

WHERE Num_BusinessUnit = 1 and client_Number is not null
and (Date_Sales >='2023-04-01' and Date_Sales <'2023-07-01') and gmv_type = "1P"
"""

df_sd_23_2_1p_q = client_1.query(query=query_global_6)
df_sd_23_2_1p = df_sd_23_2_1p_q.to_dataframe()

In [ ]:
# sales data 2023 3/3, 1P
query_global_7 = """
SELECT sales.Num_Entity as store_id, sales.Date_Sales as cart_date, sales.Num_Product as item_id, sales.Num_ticket as cart_id, 
sales.Nbr_Checkouts as nb_checkouts, 
sales.Num_SalesChannel as sale_channel_id, sales.Num_DeliveryChannel as delivery_channel_id, 
sales.Amt_GrossMerchandiseVolume as amount_to, sales.Qty_Sales as qty, sales.Top_Loyaltycard as top_l_c, sales.client_Number as user_id,
sales.gmv_type as gmv_type, sales_ch.Desc_SalesChannel as sales_channel, delivery.Desc_DeliveryChannel as delivery_channel

FROM `dfdd-performances-benchmark.BenchmarkData.taGMVProductStoreDay` as sales
left join `dfdd-performances-benchmark.BenchmarkData.tdSalesChannel` as sales_ch on sales_ch.Num_SalesChannel = sales.Num_SalesChannel 
left join `dfdd-performances-benchmark.BenchmarkData.tdDeliveryChannel` as delivery on delivery.Num_DeliveryChannel = sales.Num_DeliveryChannel

WHERE Num_BusinessUnit = 1 and client_Number is not null
and Date_Sales >='2023-07-01' and gmv_type = "1P"
"""

df_sd_23_3_1p_q = client_1.query(query=query_global_7)
df_sd_23_3_1p = df_sd_23_3_1p_q.to_dataframe()

In [ ]:
# sales data 2022, 2023 3P
query_global_8 = """
SELECT sales.Num_Entity as store_id, sales.Date_Sales as cart_date, sales.Num_Product as item_id, sales.Num_ticket as cart_id, 
sales.Nbr_Checkouts as nb_checkouts, 
sales.Num_SalesChannel as sale_channel_id, sales.Num_DeliveryChannel as delivery_channel_id, 
sales.Amt_GrossMerchandiseVolume as amount_to, sales.Qty_Sales as qty, sales.Top_Loyaltycard as top_l_c, sales.client_Number as user_id,
sales.gmv_type as gmv_type, sales_ch.Desc_SalesChannel as sales_channel, delivery.Desc_DeliveryChannel as delivery_channel

FROM `dfdd-performances-benchmark.BenchmarkData.taGMVProductStoreDay` as sales
left join `dfdd-performances-benchmark.BenchmarkData.tdSalesChannel` as sales_ch on sales_ch.Num_SalesChannel = sales.Num_SalesChannel 
left join `dfdd-performances-benchmark.BenchmarkData.tdDeliveryChannel` as delivery on delivery.Num_DeliveryChannel = sales.Num_DeliveryChannel

WHERE Num_BusinessUnit = 1 and client_Number is not null
and Date_Sales >='2023-09-01' and gmv_type = "1P"
"""

df_sd_22_23_3p_q = client_1.query(query=query_global_8)
df_sd_22_23_3p = df_sd_22_23_3p_q.to_dataframe()

In [ ]:
df_sd_22_1p = df_sd_22_1_1p.append([df_sd_22_2_1p,df_sd_22_3_1p,df_sd_22_4_1p])

In [ ]:
df_sd_23_1p_3p = df_sd_23_1_1p.append([df_sd_23_2_1p,df_sd_23_3_1p, df_sd_22_23_3p])

In [ ]:
df_sd_22_1p